In [ ]:
politica = []
farandula = []
deportes = []

with open("deportes.txt") as f:
    text = f.read()
    deportes = text.split("===")

with open("farandula.txt") as f:
    text = f.read()
    farandula = text.split("===")

with open("politica.txt") as f:
    text = f.read()
    politica = text.split("===")
print(len(deportes))
print(len(farandula))
print(len(politica))


In [ ]:
print(politica)

In [26]:
import spacy
import numpy as np

In [27]:
def extract_embedding(text: str, nlp: spacy.Language) -> np.ndarray:
    doc = nlp(text)
    return doc.vector

In [ ]:
import spacy
nlp = nlp = spacy.load("es_core_news_md")

texts = farandula + politica + deportes
embeddings = np.array([extract_embedding(text, nlp) for text in texts])
embeddings[:4, :]

In [29]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
pca = PCA(n_components=3)
embeddings_3d = pca.fit_transform(embeddings)
embeddings_3d.shape

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111, projection="3d")
ax.scatter(
    embeddings_3d[:, 0],
    embeddings_3d[:, 1],
    embeddings_3d[:, 2],
)

In [ ]:
color = [2 for i in farandula] + [3 for i in politica] + [4 for i in deportes]
color

In [34]:
pca2 = PCA(n_components=2)
embeddings_2d = pca2.fit_transform(embeddings)

In [ ]:
plt.scatter(embeddings_2d[:,0], embeddings_2d[:,1],c=color)